<span style="color:green">This file scrapes the meteo for each region using the open-meteo api</span>


In [2]:
import requests
import pandas as pd
import time

url = "https://archive-api.open-meteo.com/v1/archive"

#Coordinates for the "central point" of each region
regions = {
    "Wallonia": (50.5, 4.9),
    "Flanders": (51.0, 4.5),
    "Brussels": (50.85, 4.35)
}

hourly_params = [
    "temperature_2m",
    "apparent_temperature",
    "dewpoint_2m",
    "relative_humidity_2m",
    "precipitation",
    "rain",
    "snowfall",
    "cloudcover",
    "cloudcover_low",
    "cloudcover_mid",
    "cloudcover_high",
    "windgusts_10m",
    "shortwave_radiation",
    "direct_radiation",
    "diffuse_radiation",
    "direct_normal_irradiance",
    "et0_fao_evapotranspiration"
]

for region, (lat, lon) in regions.items():
    print(f"Fetching data for {region}...")

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": "2014-01-01",
        "end_date": "2024-12-31",
        "hourly": hourly_params,
        "timezone": "Europe/Brussels"
    }
    #Delay needed due otherwise the requests after the first will fail due to limit
    time.sleep(100)
    response = requests.get(url, params=params)

    data = response.json()

    df = pd.DataFrame(data["hourly"])

    df["time"] = pd.to_datetime(df["time"])
    
    filename = f"{region}_hourly_weather_2014_2024.csv"
    df.to_csv(filename, index=False)

    print(f"Saved {region} data to {filename}")


Fetching data for Wallonia...
Saved Wallonia data to Wallonia_hourly_weather_2014_2024.csv
Fetching data for Flanders...
Saved Flanders data to Flanders_hourly_weather_2014_2024.csv
Fetching data for Brussels...
Saved Brussels data to Brussels_hourly_weather_2014_2024.csv


In [3]:
import pandas as pd

wallonie = pd.read_csv("wallonia_hourly_weather_2014_2024.csv")
flanders = pd.read_csv("Flanders_hourly_weather_2014_2024.csv")
brussels = pd.read_csv("Brussels_hourly_weather_2014_2024.csv")

wallonie['Region'] = 1
flanders['Region'] = 2
brussels['Region'] = 3

df = pd.concat([wallonie, flanders, brussels])

df["time"] = pd.to_datetime(df["time"])
df["date"] = df["time"].dt.strftime("%Y-%m-%d")
df["hour"] = df["time"].dt.strftime("%H:%M:%S")
df.drop(columns=["time"], inplace=True)


df.to_csv("belgium_hourly_weather_2014_2024.csv", index=False)
print("Saved Belgium data to belgium_hourly_weather_2014_2024.csv")

Saved Belgium data to belgium_hourly_weather_2014_2024.csv
